In [5]:
import pandas as pd
import numpy as np

from script import cleaner

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

### Put your data in the data folder and name it 'test.csv'

In [6]:
train = pd.read_csv("./data/train.csv")
store = pd.read_csv("./data/store.csv")

/tmp/ipykernel_296855/400903536.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("./data/train.csv")


In [ ]:
test = pd.read_csv("./data/test.csv") 

In [8]:
df = cleaner(test, store)

X = df.drop(columns=['Sales'])
y = df.Sales

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the column transformer for different preprocessing steps
numeric_features = ['Competition_Since_X_months', 'weeks_since_promo2', 'Customers', 'CompetitionDistance']
one_hot_features = ['Promo', 'Promo2','DayOfWeek',  'StoreType', 'Assortment', 'PromoInterval']


numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

one_hot_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder())
])


preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('ohe', one_hot_transformer, one_hot_features)
])

In [9]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(max_depth=10, min_samples_split=2, n_estimators=150)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', rf) ])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Competition_Since_X_months',
                                                   'weeks_since_promo2',
                                                   'Customers',
                                                   'CompetitionDistance']),
                                                 ('ohe',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder())]),
                                                  ['Promo', 'Promo2',
                                                   'DayOfWeek', 'StoreType',
                                                   'Assortment',
                                                   'PromoInterval'])])),
                ('regressor',
                 RandomForestRegressor(max_depth=10, n_estimators=150))])

In [10]:
# Evaluate the model on the test data
score = pipeline.score(X_test, y_test)
print(f"Random Forest - R-squared on test data:", score)

RAndom Forest - R-squared on test data: 0.9004734920784458


In [11]:
# Get predictions for the test data
y_pred = pipeline.predict(X_test)
y_pred[:10]

In [30]:
# function to measure the performance of the model
import numpy as np

def metric(preds, actuals):
    assert preds.shape == actuals.shape
    return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])

In [32]:
# get the root mean square percentage error
rmspe = metric(y_pred, y_test.values)
rmspe

/tmp/ipykernel_293577/1119033358.py:5: RuntimeWarning: divide by zero encountered in divide
  return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])
/tmp/ipykernel_293577/1119033358.py:5: RuntimeWarning: invalid value encountered in divide
  return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])


nan